## 多重共线性的定义
对多元线性回归模型Y＝k0＋k1X1＋k2X2＋…＋knXn而言，如果特征变量X1、X2、X3…之间存在高度线性相关关系，则称为多重共线性（multicollinearity）。例如，X1＝1-X2（如：性别_男＝1-性别_女），此时X1与X2存在高度的线性相关关系，则认为该模型存在多重共线性，需要删去X1和X2中的一个变量。

如果是多个特征变量，则多重共线性可以表示成如下公式。a1X1＋a2X2＋…＋anXn＝0如果存在ai不全为0，即某个特征变量可以用其他特征变量的线性组合表示，则称特征变量间存在完全共线性。一个极端的例子是让所有ai都为1，则X1＝-（X2＋X3＋…＋Xn），此时便认为特征变量间存在完全共线性。

除了完全共线性，还存在近似共线性，它也是多重共线性的一种情况，其公式如下。a1X1＋a2X2＋…＋anXn＋v＝0如果存在ai不全为0，v为误差随机项，则称特征变量间存在近似共线性。一个极端的例子是让所有ai都为1，v为-1，那么X1＝1-（X2＋X3＋…＋Xn），此时便认为特征变量间存在近似共线性，如果n为2，那么就是之前提到的X1＝1-X2。

以一个现实中的回归模型为例：假设目标变量Y代表信用卡额度，特征变量X1代表月收入，回归模型如下。Y＝X1＋5000特征变量X1的系数为1，这表明收入增加1个单位时，信用卡额度也会增加1个单位。如果在该回归模型中加入另一个特征变量X2，其样本数据恰好与X1完全相同，那么该回归模型有可能会变成如下形式。Y＝0.5X1＋0.5X2＋5000特征变量X1的系数从1变成0.5，这表明收入增加1个单位时，信用卡额度只会增加0.5个单位，与实际情况不符，某种程度上削弱了月收入的特征重要性。因此，多重共线性会对回归模型的预测结果产生不利的影响。

**总体来说，在实际应用中，多重共线性会带来如下不利影响：**
* 线性回归估计式变得不确定或不精确；
* 线性回归估计式方差变得很大，标准误差增大；
* 当多重共线性严重时，甚至可能使估计的回归系数符号相反，得出错误的结论；
* 削弱特征变量的特征重要性。

##  多重共线性的分析与检验

In [1]:
# 读入多重线性的测试数据
import pandas as pd
df = pd.read_excel('datasets/多重线性数据.xlsx')
df.head()

,X1,X2,X3,Y
0,8,16,-32,77
1,7,14,-31,52
2,4,9,-12,42
3,1,2,5,6
4,1,2,8,19


Y列为目标变量（即因变量），X1、X2、X3列为特征变量（即自变量）

In [2]:
# 对数据集划分特征变量和目标变量
x = df.drop(columns='Y')
y = df['Y']

### 1、相关系数判断
多重共线性是指不同特征变量间存在线性相关关系，在Python中用corr()函数可以快速计算不同变量间的相关系数

In [3]:
x.corr()

,X1,X2,X3
X1,1.000000,0.992956,-0.422788
X2,0.992956,1.000000,-0.410412
X3,-0.422788,-0.410412,1.000000


其中第i行第j列的内容表示第i个特征变量和第j个特征变量的相关系数，例如，第1行第2列的相关系数0.99，表示的就是特征变量X1和特征变量X2的相关系数，可以看到它们的相关性还是非常强的，有理由相信它们会导致多重共线性，因此需要删去其中一个特征变量。需要说明的是，从左上角至右下角的对角线上的相关系数都为1，这个1其实没有什么意义，因为它表示的是特征变量自身与自身的相关系数。

这里用到的相关系数其实是皮尔逊相关系数，相关系数判断使用起来非常简单，结论也比较清晰，不过它有一个缺点：简单相关系数只是多重共线性的充分条件，不是必要条件。在有多个特征变量时，相关系数较小的特征变量间也可能存在较严重的多重共线性。为了更加严谨，实战中还经常用到下面要讲解的方差膨胀系数法（VIF检验）。

### 2、方差膨胀系数法（VIF检验）
方差膨胀系数（Variance Inflation Factor）的计算公式如下 ：$VIF_i=\frac {1}{1-R_i^2} $

其中VIFi是衡量自变量Xi是否与其他自变量具有多重共线性的方差膨胀系数；$R_i^2$是将自变量Xi作为因变量，其他自变量作为特征变量时回归的可决系数，即R-squared值，它是用来衡量拟合程度的。$R_i^2$越大，VIFi就越大，表示自变量Xi与其他自变量间的多重共线性越严重。一般认为VIFi＜10时，该自变量与其余自变量之间不存在多重共线性；当10≤VIFi＜100时存在较强的多重共线性；当VIFi≥100时存在严重的多重共线性。例如，对于只有两个特征变量的回归方程y＝f（X1，X2），若X1＝X2，根据3.2.2小节讲解的R-squared值的知识，X1＝X2即完全线性拟合，那么衡量拟合程度的可决系数$R_i^2$＝1，则方差膨胀系数VIF1＝1/（1-1）＝＋∞，大于100，即存在严重的多重共线性。

使用Python的VIF检验模块来验证特征变量间是否存在严重的多重共线性。

In [5]:
from statsmodels.stats.outliers_influence import variance_inflation_factor # 从statsmodels模块引入variance_inflation_factor()函数
vif = []
for i in x.columns: # i对应每一列的列名
    vif.append(variance_inflation_factor(x.values, x.columns.get_loc(i))) # 通过for循环依次求得每个特征变量的方差膨胀系数并将结果放入列表中
vif

[259.6430487184967, 257.6315718292196, 1.302330632715429]

因为特征变量X2是X1的2倍，所以使用X1对X2和X3回归和使用X2对X1和X3回归时所得的方差膨胀系数会很大，从上述计算结果也可以看出，前2个VIF值均大于100，暗示多重共线性十分严重，应该删掉X1或X2。

**删掉X2再进行一次回归和VIF检验，看看结果的变化**

In [7]:
# 对数据集划分特征变量和目标变量
x = df[['X1', 'X3']]
y = df['Y']

# 进行VIF检验
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(x.values, x.columns.get_loc(i))
       for i in x.columns] # 等同于上面的写法，通过for循环依次求得每个特征变量的方差膨胀系数并将结果放入列表中，该行代码可以看成求VIF值的固定写法，其中X.columns.get_loc（i）返回的是指定列的列序号数字，如第1列返回的就是数字0
vif

[1.289349054516766, 1.289349054516766]

此时两个特征变量的方差膨胀系数都小于10，说明它们之间不存在多重共线性。

**总结来说**，对于线性回归模型和逻辑回归模型等以线性方程表达式为基础的机器学习模型，需要注意多重共线性的影响。如果存在多重共线性，则需要进行相应处理，如删去某个引起多重共线性的特征变量。